In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import polars as pl
import util

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
# trip data
hh = pd.read_csv(util.output_path / 'agg/dash/hh_geog.csv')
# person data
person = pd.read_csv(util.output_path / 'agg/dash/pptyp_county.csv')
# worker data
employment = pd.read_csv(util.output_path / 'agg/dash/person_worker_type.csv')
# network data
df_network = util.process_network_summary()
# job data
df_lu = util.get_parcels_urbansim_data(inc_geog=True)


In [3]:
county_order = ['King','Kitsap','Pierce','Snohomish','Outside Region','Total']

person['person_county'] = pd.Categorical(person['person_county'], ordered=True,
                   categories=county_order)
hh['hh_county'] = pd.Categorical(hh['hh_county'], ordered=True,
                   categories=county_order)
employment['person_work_county'] = pd.Categorical(employment['person_work_county'], ordered=True,
                   categories=county_order)

df_network['county'] = pd.Categorical(df_network['county'], ordered=True,
                   categories=county_order)

In [4]:
#| label: total_vmt_vht_delay_by_county

df_vmt = df_network.reset_index().groupby('county',observed=True)['VMT'].sum()
df_vht = df_network.reset_index().groupby('county',observed=True)['VHT'].sum()
df_delay = df_network.reset_index().groupby('county',observed=True)['total_delay'].sum()

df = pd.concat([df_vmt,df_vht,df_delay], axis=1)
df = df[df.index!='Outside Region']
df.rename(columns={'total_delay': 'Total Delay Hours'}, inplace=True)

df.loc['Total',] = df.sum()
total_vmt = df.loc['Total','VMT']

# df.style.format('{:,.0f}')

## Households, Population and Vehicles

In [5]:
auto_ownership = pd.read_csv(util.output_path / 'agg/dash/auto_ownership.csv')
df_vehicle = auto_ownership.groupby('hh_county', observed=True).apply(lambda x: (x['hhvehs'] * x['hhexpfac']).sum(),include_groups=False)
df_vehicle.name = 'Vehicles'

In [6]:
# Total Population
df_person = person[['person_county','psexpfac']].groupby('person_county', observed=True)['psexpfac'].sum()
df_person.name = 'Persons'
df_hh = hh[['hh_county','hhexpfac']].groupby('hh_county', observed=True)['hhexpfac'].sum()
df_hh.name = 'Households'

df = pd.concat([df_person,df_hh,df_vehicle],axis=1)
df = df[df.index!='Outside Region']

df.loc['Total',] = df.sum()

df

,Persons,Households,Vehicles
King,"2,988,576","1,288,286","1,905,840"
Kitsap,"354,278","150,087","284,954"
Pierce,"1,237,605","500,664","872,066"
Snohomish,"1,207,372","482,015","856,924"
Total,"5,787,831","2,421,052","3,919,784"


## Workers and Jobs
By Workplace Location

Workers are Daysim outputs and Jobs are Parcel Inputs

In [7]:
# Jobs
df_jobs = df_lu[['emptot_p','CountyName']].groupby('CountyName').sum()[['emptot_p']]
df_jobs = df_jobs.reset_index()
df_jobs.rename(columns={'emptot_p': 'Jobs',
                   'CountyName':'County'}, inplace=True)
df_jobs.set_index('County')
df_jobs['Jobs'] = df_jobs['Jobs'].astype('float')

# workers
df = employment.groupby('person_work_county',observed=True)['psexpfac'].sum()

df = df.reset_index()
df.rename(columns={'psexpfac': 'Workers',
                   'person_work_county':'County'}, inplace=True)
df.set_index('County')
df['Workers'] = df['Workers'].astype('float')
df = df.merge(df_jobs, on='County')

df.loc['Total',:] = df.sum(axis=0, numeric_only=True)

df.loc['Total','County'] = 'Region'
df.reset_index(drop=True)

,County,Workers,Jobs
0,King,"1,847,733","2,048,436"
1,Kitsap,"144,156","147,104"
2,Pierce,"476,045","472,355"
3,Snohomish,"484,229","493,145"
4,Outside Region,1,0
5,Region,"2,952,164","3,161,040"


## VMT by Type

In [8]:
vmt_df = pd.DataFrame()

# Daysim Resident Demand
df = pd.read_csv(util.output_path / 'agg/dash/person_vmt.csv')
df = df[df['mode'].isin(['SOV','HOV2','HOV3+']) & df['dorp']==1]
vmt_df.loc['Resident','VMT'] = df['travdist_wt'].sum()

# Trucks
df_network['medium_truck_vmt'] = df_network['@medium_truck']*df_network['length']
df_network['heavy_truck_vmt'] = df_network['@heavy_truck']*df_network['length']

vmt_df.loc['Medium Truck','VMT'] = df_network['medium_truck_vmt'].sum()
vmt_df.loc['Heavy Truck','VMT'] = df_network['heavy_truck_vmt'].sum()

# Externals
external_vmt = total_vmt - vmt_df['VMT'].sum()
vmt_df.loc['External and Other'] = external_vmt

vmt_df.loc['Total'] = total_vmt
vmt_df

,VMT
Resident,"73,439,316"
Medium Truck,"5,362,460"
Heavy Truck,"7,101,250"
External and Other,"12,405,686"
Total,"98,308,712"


## Vehicle Trips (Resident + External)

In [9]:
# Trips
df = pd.read_csv(util.output_path / 'trips_by_class.csv')
df.columns = ['class','trips','tod']
df['mode'] = df['class'].map({'sov_inc1': 'Drive Alone',
                 'sov_inc2': 'Drive Alone',
                 'sov_inc3': 'Drive Alone',
                 'hov2_inc1': 'Shared Ride',
                 'hov2_inc2': 'Shared Ride',
                 'hov2_inc3': 'Shared Ride',
                 'hov3_inc1': 'Shared Ride',
                 'hov3_inc2': 'Shared Ride',
                 'hov3_inc3': 'Shared Ride'})
df = df.groupby('mode').sum()[['trips']].reset_index()
df

,mode,trips
0,Drive Alone,"9,197,109"
1,Shared Ride,"3,889,537"


## Emissions
Daily Tons for light, medium, and heavy vehicles; bus vehicles are excluded.

In [10]:
df = pd.read_csv(util.output_path / 'emissions/emissions_summary.csv')

df = df[df['veh_type'].isin(['light','medium','heavy'])]
df = df.groupby('pollutant_name').sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'},
                  inplace=True)
df = df.loc[df.index.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent']),
            ['Start', 'Intrazonal', 'Interzonal', 'Total Daily']]

df

,Start,Intrazonal,Interzonal,Total Daily
pollutant_name,,,,
CO,7,0,21,28
CO2 Equivalent,164,23,"6,353","6,540"
NOx,1,0,4,4
PM10 Total,0,0,5,5
PM25 Total,0,0,1,1


## Lane Miles

In [11]:
facility_type_dict = {
    1: 'Interstate',
    2: 'Other Freeway',
    3: 'Expressway',
    4: 'Ramp',
    5: 'Principal Arterial',
    6: 'Minor Arterial',
    7: 'Major Collector',
    8: 'Minor Collector',
    9: 'Local',
    10: 'Busway',
    11: 'Non-Motor',
    12: 'Light Rail',
    13: 'Commuter Rail',
    15: 'Ferry',
    16: 'Passenger-Only Ferry',
    17: 'Centroid Connector',
    18: 'Facility Connector',
    19: 'HOV Only Freeway',
    20: 'HOV Only Ramp',
    98: 'Weave Links'
}

ul3_dict = {
    0: 'Rail/Walk/Ferry',
    1: 'Freeway',
    2: 'Expressway',
    3: 'Urban Arterial',
    4: 'One-way Arterial',
    5: 'Centroid Connector',
    6: 'Rural Arterial'
}

county_dict = {
    33: 'King',
    35: 'Kitsap',
    53: 'Pierce',
    61: 'Snohomish'
}

In [12]:
# Select mid-day network
gdf = df_network.loc[df_network['tod'] == '10to14'].copy()
gdf['Lane Miles'] = gdf['length'] * gdf['num_lanes']

gdf['Facility Type'] = pd.Categorical(gdf['@facilitytype'].map(facility_type_dict), 
                                      ordered=True,
                                      categories=facility_type_dict.values())

df = gdf.groupby(['Facility Type'], observed=True)['Lane Miles'].sum().reset_index().set_index('Facility Type')

# df = df.drop(['@facilitytype','Facility Type'], axis=1)
df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
Facility Type,
Interstate,"1,533"
Other Freeway,862
Expressway,242
Ramp,550
Principal Arterial,"3,167"
Minor Arterial,"4,081"
Major Collector,"2,927"
Minor Collector,221
Local,307


In [13]:
gdf['Facility Group'] = pd.Categorical(gdf['data3'].map(ul3_dict), 
                                      ordered=True,
                                      categories=ul3_dict.values())

df = gdf.groupby(['Facility Group'], observed=True)['Lane Miles'].sum().reset_index().set_index('Facility Group')

df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
Facility Group,
Freeway,"2,396"
Expressway,707
Urban Arterial,"6,920"
One-way Arterial,454
Centroid Connector,"9,851"
Rural Arterial,"3,384"
Total,"23,711"


In [14]:
# Load link attributes and join
gdf['county'] = gdf['county'].replace(np.nan,'Outside Region')
df = gdf.groupby(['county'], observed=True)['Lane Miles'].sum().reset_index().set_index('county')
df = df[df.index!="Outside Region"]

df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
county,
King,"10,903"
Kitsap,"2,037"
Pierce,"5,946"
Snohomish,"4,817"
Total,"23,703"


# Person Metrics

In [15]:
# Daysim data

trip = pl.read_csv(util.output_path / 'daysim/_trip.tsv',separator='\t')
person = pl.read_csv(util.output_path / 'daysim/_person.tsv',separator='\t')
hh = pl.read_csv(util.output_path / 'daysim/_household.tsv', separator='\t')

### Average Daily Miles Driven per Person

In [16]:
if 'sov_ff_time' in trip.columns:
    drive_modes = [3, 4, 5]
    vehicle_trips = trip[['mode', 'dorp', 'travtime', 'sov_ff_time', 'travdist']].filter(pl.col("mode").is_in(drive_modes)).filter(pl.col("dorp") == 1)
    avg_vmt = vehicle_trips['travdist'].sum()/ person['psexpfac'].sum()
    print(f'Average Daily VMT per person, not including externals or trucks: {avg_vmt:.1f}')

Average Daily VMT per person, not including externals or trucks: 12.7


### Hours of Congestion per Person per Year
For average Puget Sound resident:

In [17]:
if 'sov_ff_time' in trip.columns:
    drive_trips = trip[['mode', 'dorp', 'travtime', 'sov_ff_time', 'travdist']].filter(pl.col("mode").is_in(drive_modes))
    drive_trips = drive_trips.with_columns(
        delay=pl.col('travtime')-(pl.col('sov_ff_time')/100.0))
    minutes_to_hour = 60
    drive_mode_delay = util.summary_config['weekday_to_annual']*(drive_trips['delay'].sum()/person['psexpfac'].sum())/minutes_to_hour
    print(f'Annual hours of delay for residents, not including externals or trucks is: {drive_mode_delay:.1f}')

Annual hours of delay for residents, not including externals or trucks is: 16.2


### Annual Hours of Delay by Average Truck
Average annual delay (hours) per truck trip in and through the region:

In [18]:
# Total truck trips

In [19]:
# Load truck trips
df = pd.read_csv(util.output_path / 'trucks/trucks_summary.csv',index_col=0)

# Truck delay
net_sum = pd.read_csv(util.output_path / 'network/delay_user_class.csv')

# Annual delay hours
daily_delay = net_sum[['@mveh','@hveh']].sum().sum()


# total truck trips
trips = df['prod'].sum()

# average annual delay hours per truck
x = (daily_delay*util.summary_config["weekday_to_annual"])/trips
print('{:0,.1f}'.format(x))

22.5


*Medium trucks only:*

In [20]:
x = (net_sum['@mveh'].sum()*util.summary_config["weekday_to_annual"])/df.loc['mt','prod']
print('{:0,.1f}'.format(x))

14.6


*Heavy trucks only:*

In [21]:
x = (net_sum['@hveh'].sum()*util.summary_config["weekday_to_annual"])/df.loc['ht','prod']
print ('{:0,.1f}'.format(x))

47.8


## % Population Walking or Biking for Transportation

In [22]:
trip_person = trip.join(person, on=["hhno", "pno"], how="left")
bike_walk_trips = trip_person.filter(pl.col("mode").is_in([1, 2]))

# Get unique persons with at least one bike/walk trip
bike_walk_persons = bike_walk_trips.select(["hhno", "pno"]).unique()
bike_walk_persons = bike_walk_persons.with_columns(bike_walk=pl.lit(True))

# Join back to all persons, mark bike_walk as False if not present
person_with_bike_walk = person.join(bike_walk_persons, on=["hhno", "pno"], how="left")
person_with_bike_walk = person_with_bike_walk.with_columns(
    pl.col("bike_walk").fill_null(False)
)

# Calculate share
share = (
    person_with_bike_walk.group_by("bike_walk")
    .agg(pl.col("psexpfac").sum())
    .with_columns(
        (pl.col("psexpfac") / pl.col("psexpfac").sum()).alias("share")
    )
)
print("Percent of population with at least one non-exercise walk or bike trip: {:.1%}".format(share.filter(pl.col("bike_walk") == True)["share"][0]))

Percent of population with at least one non-exercise walk or bike trip: 32.2%


## Household and Jobs within 1/4 mile transit

In [23]:
# Network data
df = pd.read_csv(util.output_path / 'transit/transit_access.csv',index_col=0)

**Households**

In [24]:
x = df.loc['hh_p','quarter_mile_transit']
print('{:,.0f}'.format(x) + (" households within 1/4 mile of transit"))
x = df.loc['hh_p','quarter_mile_transit']/df.loc['hh_p','total']
print('{:,.1%}'.format(x) + (" of total households"))

1,372,604 households within 1/4 mile of transit
56.7% of total households


**Jobs**

In [25]:
x = df.loc['emptot_p','quarter_mile_transit']
print('{:,.0f}'.format(x) + (" jobs within 1/4 mile of transit"))
x = df.loc['hh_p','quarter_mile_transit']/df.loc['emptot_p','total']
print('{:,.1%}'.format(x) + (" of total jobs"))

2,384,286 jobs within 1/4 mile of transit
43.4% of total jobs
